In [ ]:
import openpyxl
import requests

def translate_text_deepl(text, source_lang, target_lang, auth_key):
    url = "https://api-free.deepl.com/v2/translate"
    headers = {
        "Authorization": f"DeepL-Auth-Key {auth_key}"
    }
    data = {
        "text": text,
        "source_lang": source_lang,
        "target_lang": target_lang
    }

    response = requests.post(url, headers=headers, data=data)
    if response.ok:
        translation = response.json()["translations"][0]["text"]
        return translation
    else:
        print("Translation failed. Status code:", response.status_code)
        return None

def is_valid_auth_key(auth_key):
    url = "https://api-free.deepl.com/v2/usage"
    headers = {
        "Authorization": f"DeepL-Auth-Key {auth_key}"
    }

    response = requests.get(url, headers=headers)
    return response.ok

# Replace "[yourAuthKey]" with your actual DeepL API authentication key
deepl_auth_key = "[yourAuthKey]"
source_language = "TR"
target_language = "EN"

# Load the XLSX file
file_path = "dosya.xlsx"
workbook = openpyxl.load_workbook(file_path)

# Check if the DeepL API key is valid
if not is_valid_auth_key(deepl_auth_key):
    print("Invalid DeepL API key. Please check your authentication key.")
    exit()

# Translate the sheet names in the workbook
for sheet_name in workbook.sheetnames:
    translated_sheet_name = translate_text_deepl(sheet_name, source_language, target_language, deepl_auth_key)
    if translated_sheet_name:
        workbook[sheet_name].title = translated_sheet_name

# Translate all text values in the workbook
for sheet in workbook:
    for row in sheet.iter_rows(values_only=True):
        for cell_value in row:
            if isinstance(cell_value, str):
                translated_text = translate_text_deepl(cell_value, source_language, target_language, deepl_auth_key)
                if translated_text:
                    cell = sheet.cell(row=row[0].row, column=row[0].column + row.index(cell_value))
                    cell.value = translated_text

# Save the translated XLSX file to a new filename
translated_file_path = "translated_dosya.xlsx"
workbook.save(translated_file_path)
